In [53]:
import pandas as pd
from nltk.corpus import stopwords
import string
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.svm import SVC
from gensim.models import Word2Vec,KeyedVectors
import os
import wget
import gzip
import shutil
import numpy as np

In [2]:
df=pd.read_csv(r'all-data.csv',encoding = "ISO-8859-1",header=None)

In [3]:
df.head()

,0,1
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [4]:
df.columns=['label','texts']

In [5]:
df.head()

,label,texts
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [6]:
df.label.value_counts()/df.shape[0]

neutral     0.594098
positive    0.281263
negative    0.124639
Name: label, dtype: float64

In [14]:
stop_words=stopwords.words("english")
def preprocess(df):
#     function to remove stopwords,digts and punctions  
    df['texts']=df['texts'].apply(lambda x:" ".join(x.lower() for x in x.split()))
    df['texts']=df['texts'].apply(lambda x:" ".join(x for x in x.split() if x not in stop_words))
    df['texts']=df['texts'].apply(lambda x:" ".join(x for x in x.split() if not x.isdigit()))
    df['texts']=df['texts'].str.replace('[^\w\s]','')

In [15]:
preprocess(df)

In [16]:
df

,label,texts
0,0,according gran company plans move production r...
1,0,technopolis plans develop stages area less squ...
2,2,international electronic industry company elco...
3,1,new production plant company would increase ca...
4,1,according company updated strategy years baswa...
...,...,...
4841,2,london marketwatch share prices ended lower lo...
4842,0,rinkuskiai beer sales fell per cent million li...
4843,2,operating profit fell eur mn eur mn including ...
4844,2,net sales paper segment decreased eur mn secon...


In [17]:
# df['label']=df.label.map({'neutral':0,'positive':1,"negative":2})

In [18]:
df.head()

,label,texts
0,0,according gran company plans move production r...
1,0,technopolis plans develop stages area less squ...
2,2,international electronic industry company elco...
3,1,new production plant company would increase ca...
4,1,according company updated strategy years baswa...


In [19]:
x=df['texts']
y=df['label']
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=12,test_size=0.3)

In [20]:
vec=CountVectorizer()
x_train_vec=vec.fit_transform(x_train)
x_test_vec=vec.transform(x_test)

In [23]:
nb=MultinomialNB()
nb.fit(x_train_vec,y_train)

MultinomialNB()

In [26]:
nb_y_pred=nb.predict(x_test_vec)

In [27]:
confusion_matrix(nb_y_pred,y_test)

array([[711, 158,  73],
       [114, 217,  51],
       [ 16,  32,  82]], dtype=int64)

In [28]:
accuracy_score(nb_y_pred,y_test)

0.6946354883081155

In [31]:
svc=SVC()
svc.fit(x_train_vec,y_train)
svc_y_pred=svc.predict(x_test_vec)

In [32]:
confusion_matrix(svc_y_pred,y_test)

array([[818, 236, 111],
       [ 22, 157,  40],
       [  1,  14,  55]], dtype=int64)

In [33]:
accuracy_score(svc_y_pred,y_test)

0.7083906464924347

In [38]:
file_zip_path=r'D:\jupyternotebooks\Untitled Folder\GoogleNews-vectors-negative300.bin.gz'
file_vec_path=r'D:\jupyternotebooks\Untitled Folder\GoogleNews-vectors-negative300.bin'
with gzip.open(file_zip_path,'rb') as f_in:
    with open(file_vec_path,'wb') as f_out:
        shutil.copyfileobj(f_in,f_out)

In [39]:
w2v_model=KeyedVectors.load_word2vec_format(file_vec_path,binary=True)

In [41]:
w2v_model_keys=w2v_model.vocab.keys()
w2v_lower=[item.lower() for item in w2v_model_keys]

In [54]:
def list_of_tokens(df):
    list_of_list=[]
    for index,row in df.iterrows():
        row_text=row['texts']
        list_of_list.append(row_text.split())
    return list_of_list

def embedding(list_of_list):
    DIMENSION=300
    zero_vector=np.zeros(DIMENSION)
    feats=[]
    for tokens in list_of_list:
        feat_for_this=np.zeros(DIMENSION)
        count_for_this=0+ 1e-5
        for token in tokens:
            if token in w2v_model:
                feat_for_this+=w2v_model[token]
                count_for_this+=1
        if(count_for_this!=0):
            feats.append(feat_for_this/count_for_this)
        else:
            feats.append(zero_vector)
    return feats

In [56]:
train_vectors=embedding(list_of_tokens(df))

In [57]:
x_train,x_test,y_train,y_test=train_test_split(train_vectors,y,random_state=12,test_size=0.3)

In [61]:
svc=SVC()
svc.fit(x_train,y_train)
svc_y_pred=svc.predict(x_test)

In [62]:
accuracy_score(svc_y_pred,y_test)

0.750343878954608